In [6]:
import arcpy


In [8]:
# Input layer name
input_layer = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\TaxPlat_No_P1\TaxPlat_No_P1.shp"

# List of fields to add
fields_to_add = ["OSDS", "Footprint", "Mains"]

# Add fields to the layer
for field in fields_to_add:
    try:
        arcpy.AddField_management(input_layer, field, "TEXT")  # Add text field
        print(f"Successfully added field: {field}")
    except Exception as e:
        print(f"Failed to add field {field}: {e}")

# List of layers corresponding to each field
intersect_layers = {
    "OSDS": r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\OSDS\OahuOSDS\On-site_Sewage_Disposal_Systems_-_Oahu.shp",  # Replace with the path to the OSDS layer
    "Footprint": r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Building_Footprint_Centerpoints\Building_Footprint_Centerpoints.shp",  # Replace with the path to the Footprint layer
    "Mains": r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\SewerMains\OahuSewerMains\Sewer_Main.shp",  # Replace with the path to the Mains layer
}

# Update cursor to update the fields based on intersection
with arcpy.da.UpdateCursor(input_layer, ["OID@"] + fields_to_add) as cursor:
    for row in cursor:
        oid = row[0]
        row_update = [oid]  # Start with the OID

        # Get the geometry of the current feature in the input layer
        where_clause = f"OBJECTID = {oid}"
        with arcpy.da.SearchCursor(input_layer, ["SHAPE@"], where_clause) as search_cursor:
            geometry = None
            for search_row in search_cursor:
                geometry = search_row[0]
                break  # Only expect one row per OBJECTID

        if geometry:
            for field, layer_path in intersect_layers.items():
                intersects = False
                with arcpy.da.SearchCursor(layer_path, ["SHAPE@"]) as intersect_cursor:
                    for intersect_row in intersect_cursor:
                        intersect_geometry = intersect_row[0]
                        if geometry.overlaps(intersect_geometry) or geometry.intersects(intersect_geometry):
                            intersects = True
                            break
                row_update.append("yes" if intersects else "no")
        else:
            row_update.extend(["no", "no", "no"])  # If no geometry found, default to "no" for all fields

        # Update the row
        row[1:] = row_update[1:]  # Exclude the OID from update
        cursor.updateRow(row)

print("Field update process complete.")

Successfully added field: OSDS
Successfully added field: Footprint
Successfully added field: Mains


AttributeError: 'Polygon' object has no attribute 'intersects'